# Getting Started with BigQuery 

Google BigQuery is a managed, scalable data warehouse that enables analysis over large sets of data.  BigQuery excels in querying and processing very large data sets in a SQL like fashion.  In this lab we will explore how easy it is to use and drive insights from our data using a Python Notebook with BigQuery!

#### Lab: Querying from a Public Dataset
In this lab we will be querying from one of the many [publicly available BigQuery Datasets.](https://cloud.google.com/bigquery/public-data)  Our code blocks below uses `%%bigquery` annotation which magically queries BigQuery and returns data as a Pandas Dataframe.  You can also [use the BigQuery UI as well](https://console.cloud.google.com/bigquery?authuser=1)

Run the code block below to execute our first query and view weather data from the NOAA.

In [ ]:
%%bigquery
SELECT * FROM
  `bigquery-public-data.noaa_gsod.gsod*` LIMIT 10

This data is useful, but it would be better enriched if we knew the station id's proper name.  The below code block demonstrates joining two data sets together to get a result

In [ ]:
%%bigquery
SELECT a.name, a.state, a.country, b.*
FROM `fh-bigquery.weather_gsod.stations` a
JOIN `bigquery-public-data.noaa_gsod.gsod*` b
ON a.usaf=b.stn AND a.wban=b.wban LIMIT 10;

Lastly during this lab, to emphasize how big our data sets are, we will do a count on the dataset.  Run the code block below to see how quickly BigQuery can handle very large datasets.

In [ ]:
%%bigquery
SELECT count(1) FROM
  `bigquery-public-data.noaa_gsod.gsod*`


#### Lab: Gaining Insights From Our Data
In this section we will perform some advanced querying to find the average daily temperature by month, for a specific municipality.  We will use AVG() and GROUP BY to find the average. 

In [ ]:
%%bigquery
SELECT a.name, b.mo, AVG(temp) as Temp_f
FROM `fh-bigquery.weather_gsod.stations` a
JOIN `bigquery-public-data.noaa_gsod.gsod201*` b
ON a.usaf=b.stn AND a.wban=b.wban 
WHERE a.state = 'CA' AND a.name = 'SAN FRANCISCO INTL'
GROUP BY a.name, b.mo
ORDER BY b.mo asc;

Now its your turn! Change the state and name to a different municipality to find your own insights!
**Hint:** You can use the this query to find weather stations in a different state.
```%%bigquery avg_temperature
SELECT DISTINCT a.name
FROM `fh-bigquery.weather_gsod.stations` a
JOIN `bigquery-public-data.noaa_gsod.gsod*` b
ON a.usaf=b.stn AND a.wban=b.wban 
WHERE a.state = 'MI' LIMIT 10;
```

Next we will visualize our data right here in the Python Notebook using the magic command `%matplotlib inline`.  For this example we will find the average temperature by month for the State of Alaska. First run the BigQuery query below to save the results into a Pandas Dataframe called `avg_temperature` to be made available in the following code block.

In [ ]:
%%bigquery avg_temperature
SELECT a.state, b.mo, AVG(temp) as Temp_f
FROM `fh-bigquery.weather_gsod.stations` a
JOIN `bigquery-public-data.noaa_gsod.gsod201*` b
ON a.usaf=b.stn AND a.wban=b.wban 
WHERE a.state = 'AK'
GROUP BY a.state, b.mo
ORDER BY b.mo asc;

In [ ]:
%matplotlib inline
avg_temperature.plot(kind='bar', x='mo', y='Temp_f');


**First:** Modify the query above to find the average precipitation for Alaska (AK) throughout the year.

**Second:** Change the query above to find results for your favorite city, and state!

### Using Python to drive insights in BigQuery

BigQuery also has an extensive Python library that you can draw from. Below is an example from before this time querying the dataset using Python.
This time we will find the top 5 warmest average, and bottom 5 coolest average weather stations.

In [ ]:
import pandas as pd

from google.cloud import bigquery
client = bigquery.Client()

sql = """
SELECT a.name, AVG(temp) as Temp_f
FROM `fh-bigquery.weather_gsod.stations` a
JOIN `bigquery-public-data.noaa_gsod.gsod201*` b
ON a.usaf=b.stn AND a.wban=b.wban 
WHERE a.state = 'CA' 
GROUP BY a.name
ORDER BY Temp_f desc LIMIT 5;
"""
df_desc = client.query(sql).to_dataframe()

sql = """
SELECT a.name, AVG(temp) as Temp_f
FROM `fh-bigquery.weather_gsod.stations` a
JOIN `bigquery-public-data.noaa_gsod.gsod201*` b
ON a.usaf=b.stn AND a.wban=b.wban 
WHERE a.state = 'CA' 
GROUP BY a.name
ORDER BY Temp_f asc LIMIT 5;
"""
df_asc = client.query(sql).to_dataframe()

frames = [df_asc, df_desc]
result = pd.concat(frames)
ax = result.plot(kind='bar', x='name', y='Temp_f', figsize=(15,7))
ax.set_title('Hottest and Coldest Weather Stations in Califora')
ax.set_xlabel('Station Name')
ax.set_ylabel('Temp (F)');


BigQuery, Python and Dataframes also supports other powerful techniques such as pivot tables. Pivot tables allow a summary of the data in a more advanced format featuring averages, sums, counts, and other statistics.  This example below takes the average temperature data and displays it in both graph and table form for multiple states.

In [ ]:
import pandas as pd

from google.cloud import bigquery
client = bigquery.Client()

sql = """
SELECT AVG(b.temp) as Temp_f, b.year, a.state
FROM `fh-bigquery.weather_gsod.stations` a
JOIN `bigquery-public-data.noaa_gsod.gsod*` b
ON a.usaf=b.stn AND a.wban=b.wban 
WHERE a.country='US' AND a.state IN ('MI','CA','AK') AND b.year > '1990' AND b.year < '2020'
Group By b.year, a.state
"""

df = client.query(sql).to_dataframe()
pivot_table = pd.pivot_table(df, index=['year'], columns=['state'], values=['Temp_f'])
pivot_table.plot(kind='bar', figsize=(15, 7));
display(pivot_table)

Modify the Python code above to find your own insights in the NOAA dataset!